In [130]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import pandas as pd
import json
from bs4 import BeautifulSoup
import requests
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn import datasets, linear_model
import numpy as np

In [131]:
def arrumapreco(valor):  #função que retira todos os sinais(emocotions continuam)
    if "M" in valor:
        valor=valor.replace("M","000000")
        valor=valor.replace("€","")
        if "." in valor:
            valor=valor.replace(".","")
            return int(valor)/10000000
        else:
            return int(valor)/1000000
    
    else:
        valor=valor.replace("K","000")
        valor=valor.replace("€","")
        if "." in valor:
            valor=valor.replace(".","")
            return int(valor)/1000000
        else:
            return int(valor)/1000000

def arrumaatt(att):
    if type(att) !=int:
        if "+" in att:
            att=att[:att.find("+")]
        elif "-" in att:
            att=att[:att.find("-")]
    return att



In [132]:
#Inicializa os datasets e arruma seus valores
df=pd.read_csv('Database\CompleteDataset.csv',index_col=0)
listtwitter = json.loads(open("followers.txt").read())
dft=pd.DataFrame(listtwitter,columns=["Name","Seguidores","Twitter"])
dfall = df.merge(dft, how = 'inner', on = ['Name'])
dfall=dfall.dropna()


features=["Acceleration","Agility","Ball control","Dribbling","Long shots","Marking","Reactions","Short passing","Shot power","Sliding tackle","Stamina","Strength","Volleys"]
features=["Acceleration","Aggression","Crossing","Dribbling","Finishing","Jumping","Long shots","Reactions","Stamina","Strength"]
features2=features+["Followers"]

dfall=dfall.rename(columns = {'Preferred Positions':'Positions'})
dfall.Value=dfall.Value.apply(arrumapreco)

dfall=dfall[(dfall.Value>1)]
dfall=dfall[(dfall.Positions == 'ST LW ') | (dfall.Positions == 'RW ') | (dfall.Positions == 'LW ') | (dfall.Positions == 'ST ')| (dfall.Positions == 'CF ')| (dfall.Positions == 'ST RW ')| (dfall.Positions == 'CAM ')]

for i in features:
    dfall[i]=dfall[i].apply(arrumaatt)



C:\Users\Samuel\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (23,35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [133]:
X = dfall[features]
X2 = pd.concat([dfall[features],dfall.Seguidores],axis=1)
y = dfall.Value



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y, test_size=0.1, random_state=42)

In [134]:
regr = linear_model.LinearRegression()# cria o modelo
regr2 = linear_model.LinearRegression()

regr.fit(X_train, y_train) # treina o modelo
regr2.fit(X_train2, y_train2) # treina o modelo

xfit = X_train
yfit = regr.predict(X_train)
xfit2 = X_train2
yfit2 = regr2.predict(X_train2)

In [135]:
#OLS
yy = y_train
XX = X_train
XX = sm.add_constant(XX)

model = sm.OLS(yy, XX.astype(float)).fit()
print(model.summary())

#OLS
yy2 = y_train2
XX2 = X_train2
XX2 = sm.add_constant(XX2)

model2 = sm.OLS(yy2, XX2.astype(float)).fit()
print()
print()
print()
print()
print()
print()
print(model2.summary())



                            OLS Regression Results                            
Dep. Variable:                  Value   R-squared:                       0.751
Model:                            OLS   Adj. R-squared:                  0.734
Method:                 Least Squares   F-statistic:                     44.12
Date:                Mon, 20 Nov 2017   Prob (F-statistic):           3.29e-39
Time:                        02:36:34   Log-Likelihood:                -588.09
No. Observations:                 157   AIC:                             1198.
Df Residuals:                     146   BIC:                             1232.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         -279.3339     17.534    -15.931   

In [136]:
estim = regr.predict(X)
estim_seg = regr2.predict(X2)
dfall['Estim']=estim
dfall['Estim_Seguidores']=estim_seg
dfall[["Name","Value","Estim","Estim_Seguidores","Value","Seguidores","Twitter"]]


,Name,Value,Estim,Estim_Seguidores,Value,Seguidores,Twitter
0,Cristiano Ronaldo,95.5,81.223290,96.501806,95.5,64104546,@Cristiano
1,L. Messi,105.0,90.285185,86.708309,105.0,1424743,@messi10stats
2,Neymar,123.0,69.639627,79.455324,123.0,35163756,@neymarjr
3,L. Suárez,97.0,70.943422,71.812831,97.0,12380660,@LuisSuarez9
7,R. Lewandowski,92.0,60.677301,58.152454,92.0,981411,@lewy_official
9,E. Hazard,90.5,61.766004,61.156887,90.5,5168450,@hazardeden10
11,G. Higuaín,77.0,60.921829,59.715030,77.0,6722354,@G_Higuain
18,G. Bale,69.5,61.324298,62.058882,69.5,15693471,@GarethBale11
19,S. Agüero,66.5,65.602367,66.433898,66.5,12496406,@aguerosergiokun
26,P. Aubameyang,61.0,57.109371,53.232097,61.0,741565,@Aubameyang7
